Time to stimulate the thought processes.


There is NO NEW code here. You've seen similar code many times.
I expect that the vast majority of which has been better written.

My thoughts are scattered around the notebook. Again you generally probably see (or have) better ones. 
Mine are different in that I don't bother to spell check them!



Results tracking

V3  LGBM0 Overall Training Validation Score | Blend: 0.8569692225223414 with LB score of 0.85633

V4  LGBM0 Overall Training Validation Score | Blend: 0.8570320102774184 with LB score of 0.85623

V5  LGBM0 Overall Training Validation Score | Blend: 0.8570190414332162 with LB score of 0.85627

V6  LGBM1 Overall Training Validation Score | Blend: 0.8571127311293687 with LB score of 0.85633

V7   XGB0 Overall Training Validation Score | Blend: 0.8570048237818386 with LB score of 0.85645

V8   XGB1 Overall Training Validation Score | Blend: 0.8568621253215738 with LB score of 0.85621

V10 CATB0 Overall Training Validation Score | Blend: 0.8566436011399643 with LB score of 0.85597

V11 LGBM0 Overall Training Validation Score | Blend: 0.8569640619014858 with LB score of 0.85638

V12 LGBM0 Overall Training Validation Score | Blend: 0.8569158481801216 with LB score of 0.85639



Recap of version 12
That v11 vs v12 is starting to annoy me.
Let me double check my changes. Boolean features where changed to type 'category'. 
Boolean features where removed from Scaler. (another Q.  what happens to type catgeory within a scaler?)
Boolean features where listed as such within the params (another Q. is that not default for the datatype?)
So really simple changes and the CV score decreases by 5 times as much as the LB increases.
Ignoring the LB side of the problem doesn't help becuase I can't explain the CV side to start with.
And I really don't want to trawl through LGB source code to search for an answer. 
Always that same darn riddle. Do you trust LB or CV?


I'll need to go read ROC notebooks again :(
Wasted my time. It seems that is hasn't changed in ages. 




Onto Version 13.

So some really difficult situations as I want one and only one LGBM model.
Which is the right one to choose.


I'm going to choose V11 as I trust CV but obviously the reader can do whatever they want with whatever they may have to hand. Also I forgot to add v12 to the dataset.
I'm too lazy to add it now. You know where it is if you really want it.
But yes - notice I added a new dataset which should be public. This is a dataset containing the key oof and submission data that this notebook has generated. 
So it was effectively public already. I've just colleced and centralised it.


Only a couple days left and this notebook is about 20th on the list of public notebooks. An improvment of 0.0003 would put it in 1st place or public notebooks. 
It would also move it into top 30-100 of the real LB. I wonder how many of those tied for 30th have teh exact same copied code.

This gives us something to aim for as the vast majority of better scoring code is generally just blending and lets be honest: If your code runs in less that 20 seconds it probably isn't going to be that original.

I'm keen to try some ensemble with a twist. Too much of the standard stuff already published. In the past we sometimes tried adding a prediction set as a new feature and 
I'm going to give that a try today.





In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime
import gc

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc

import json
import optuna

import lightgbm as lgbm
import optuna.integration.lightgbm as lgbo
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt

from sklearn.metrics import roc_auc_score

from warnings import filterwarnings
filterwarnings('ignore')

from warnings import resetwarnings

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

## Better check what version we are using as I may want to make use of init_model
import lightgbm as lgbm
print (lgbm.__version__)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time
df_train = reduce_mem_usage(dt.fread('../input/tabular-playground-series-oct-2021/train.csv').to_pandas())
df_test = reduce_mem_usage (dt.fread('../input/tabular-playground-series-oct-2021/test.csv').to_pandas())

sample_submission = pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")


In [ ]:
## Always handy to know how many features we are dealing with
num_cols = [col for col in df_test.columns]  ## remember that this will include id which isn't really a feature

In [ ]:
# prepare dataframe for modeling
X = df_train.drop(columns=["id", "target"]).copy()
y = df_train["target"].copy()

test_data = df_test.drop(columns=["id"]).copy()

In [ ]:
cat_cols = X.columns[(X.nunique() < 5)]
con_cols = X.columns[(X.nunique() >= 5)]
cat_cols_indices = [X.columns.get_loc(col) for col in cat_cols]

In [ ]:
f_pred = pd.DataFrame(np.zeros(df_train.shape[0]), columns = ['pred'] )

In [ ]:

del df_test, df_train
gc.collect()


In [ ]:


scaler = StandardScaler()
X[con_cols] = reduce_mem_usage(pd.DataFrame(columns=con_cols, data=scaler.fit_transform(X[con_cols])))
test_data[con_cols] = reduce_mem_usage(pd.DataFrame(columns=con_cols, data=scaler.transform(test_data[con_cols])))


In [ ]:
# Hopefully there is nothing new above this line
def load_sp_data():
    df_train_tmp = pd.read_csv("../input/sandpit-data/v6_f_pred_train.csv")
    df_test_tmp = pd.read_csv("../input/sandpit-data/v6_f_pred_test.csv")
    X['v6']  = df_train_tmp['pred']
    test_data['v6'] = df_test_tmp['pred']

    df_train_tmp = pd.read_csv("../input/sandpit-data/v7_f_pred_train.csv")
    df_test_tmp = pd.read_csv("../input/sandpit-data/v7_f_pred_test.csv")
    X['v7']  = df_train_tmp['pred']
    test_data['v7'] = df_test_tmp['pred']
    
    df_train_tmp = pd.read_csv("../input/sandpit-data/v8_f_pred_train.csv")
    df_test_tmp = pd.read_csv("../input/sandpit-data/v8_f_pred_test.csv")
    X['v8']  = df_train_tmp['pred']
    test_data['v8'] = df_test_tmp['pred']
    
    df_train_tmp = pd.read_csv("../input/sandpit-data/v10_f_pred_train.csv")
    df_test_tmp = pd.read_csv("../input/sandpit-data/v10_f_pred_test.csv")
    X['v10']  = df_train_tmp['pred']
    test_data['v10'] = df_test_tmp['pred']
    
    df_train_tmp = pd.read_csv("../input/sandpit-data/v11_f_pred_train.csv")
    df_test_tmp = pd.read_csv("../input/sandpit-data/v11_f_pred_test.csv")
    X['v11']  = df_train_tmp['pred']
    test_data['v11'] = df_test_tmp['pred']

In [ ]:
load_sp_data ()
# As I run this I realise that maybe I could just grap v12 now as I have also included the dataset of this notebook. So I have access to anything in last saved version

df_train_tmp = pd.read_csv("../input/tps-oct-joes-sandpit/f_pred_train.csv" )
df_test_tmp = pd.read_csv("../input/tps-oct-joes-sandpit/f_pred_test.csv")
X['v12']  = df_train_tmp['pred']
test_data['v12'] = df_test_tmp['pred']

### Just in case I want it again ... in next run.
df_train_tmp.to_csv("./f_pred_train_12.csv", index=True)
df_test_tmp.to_csv("./f_pred_test_12.csv", index=True)


f_pred_cols = ['v6', 'v7', 'v8', 'v10', 'v11', "v12" ]
print (X[f_pred_cols])


In [ ]:
# okay that looks decent but I forgot what we are dealing with. 
# From the header - I get the details
# V6 LGBM1 Overall Training Validation Score | Blend: 0.8571127311293687 with LB score of 0.85633
# V7 XGB0 Overall Training Validation Score | Blend: 0.8570048237818386 with LB score of 0.85645
# V8 XGB1 Overall Training Validation Score | Blend: 0.8568621253215738 with LB score of 0.85621
# V10 CATB0 Overall Training Validation Score | Blend: 0.8566436011399643 with LB score of 0.85597
# V11 LGBM0 Overall Training Validation Score | Blend: 0.8569640619014858 with LB score of 0.85638
# V12 LGBM0 Overall Training Validation Score | Blend: 0.8569158481801216 with LB score of 0.85639

# I want one of each type so will definetly use v10
# Then I will add v11 for my lgb choice and v7 for my xgb choice
# So (double check) I should drop v6, v8, v12

# V10 is not a great model. Hopefully not my doing and rather a 'function' of catboost on the gien dataset.
# Should we include V10 or will it just be noise. Add FI later to check

X = X.drop(columns=["v6", "v8", "v12"])

### apparently you can get much further into the notebook before it reminds you to do something simialr to test :(
test_data = test_data.drop(columns=["v6", "v8", "v12"])


###
### What happens if we don't drop them and use all 6 new features ?
### What happens if we add another 10 such new features ?
### Well that always depends. I tend to think as computers of being able to use every last bit of data but there are always cases that it doens't work in your favour.
### For this partcular case .... I don't know, I haven't tried. 

### But I am 95% sure that theh winner of this comp will have used more that one model and most probably lots of models!

print (X)

In [ ]:
del scaler
gc.collect()

In [ ]:
# I still need to find where I got these params from and the try and credit the source
lgbm_params_0 = {
     'objective': 'binary',
     'n_estimators':92700,        ## way to high but I have faith in early stopping criteria
#      'importance_type': 'gain',
     'metric':'auc',
#      'boosting_type': 'gbdt',
     'boosting_type': 'goss',
     'n_jobs' : -1,
    'learning_rate': 0.01, 
    'colsample_bytree': 0.3, 
    'reg_lambda': 0.011685550612519125, 
    'reg_alpha': 0.04502045156737212,
#     Let the model know about the categorical features    
#     'categorical_feature' : cat_features,
#     'feature_name': 'auto',
    'min_child_weight': 16.843316711276092, 
    'min_child_samples': 412, 
    'num_leaves': 546, 
    'max_depth': 5, 
    'cat_smooth': 36.40200359200525, 
    'cat_l2': 12.979520035205597
    }

# https://www.kaggle.com/vishwas21/tps-oct-21-eda-modeling
# I'm not sure if that is the original source, but it is one of the earliest notebooks that mention these params
xgb_params_0 = {
    'max_depth': 6,
    'n_estimators': 19500,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.6000000000000001,
    'min_child_weight': 56.41980735551558,
    'reg_lambda': 75.56651890088857,
    'reg_alpha': 0.11766857055687065,
    'gamma': 0.6407823221122686,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'predictor': 'cpu_predictor',
#     'tree_method': 'gpu_hist',
#     'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }



xgb_params_1 = {
    "objective": "binary:logistic",
#     "learning_rate": 8e-3,
    "learning_rate": 0.003,
    "subsample": 0.6,
    "colsample_bylevel": 0.9,
    "colsample_bytree": 0.4,
    "n_estimators": 20_000,
    "max_depth": 8,
    "alpha": 64,
    "lambda": 32,
    "min_child_weight": 8,
    "importance_type": "total_gain",
    "tree_method": "hist"#,
#     "predictor": "gpu_predictor",
}

xgb_params_2 = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "tree_method": "hist",
#     "learning_rate": 0.02,
    "learning_rate": 0.001,
    "n_estimators": 20_000,
    "random_state": 12,
    "lambda": 0.003731399945310043,
    "alpha": 0.1660536107526955,
    "colsample_bytree": 0.5164889907489927,
    "subsample": 0.5869840790716806,
    "max_depth": 18,
    "min_child_weight": 142}


lgb_params = {'n_estimators': 20000,
          'max_depth': 3,
          'learning_rate': 0.003,
          'reg_alpha': 9.5,
          'reg_lambda': 8.5,
          'num_leaves': 109,
          'min_data_per_group': 133,
          'min_child_samples': 113,
          'colsample_bytree': 0.2,
          'objective': 'binary',
          'random_state': 2001,
          'metric': 'auc',}



# Taken from https://www.kaggle.com/dwoodlock/tps-oct2021-don#Modeling
## he reckons around 0.85673 on average over 2 seeds
lgbm_params_1 =  {
    'objective' : 'binary',
    'metric' : 'auc',
    'num_leaves' : 7,
#    'learning_rate' : 0.08,
    'learning_rate' : 0.01,
#     'device' : 'gpu'
    'feature_pre_filter': False, 
    'reg_alpha': 9.31403763526178, 
    'reg_lambda': 0.10613573572439,
    'num_leaves': 7,
    'colsample_bytree': 0.4, 
    'subsample': 0.8391963650875751, 
    'subsample_freq': 5, 
    'min_child_samples': 100,
#     'num_iterations': trees, #10000,
    'n_estimators': 60000
}

catb_params_0 = {
    "objective": "CrossEntropy",
    "eval_metric": "AUC",
    "grow_policy": "SymmetricTree",
    "learning_rate": 0.004,
    "n_estimators": 30000,
    "random_strength": 1.0,
    "max_bin": 128,
    "l2_leaf_reg": 0.002550319996478972,
    "max_depth": 4,
    "min_data_in_leaf": 193,
    "random_state": 42
}

In [ ]:
# ### Now what do we choose for modelling.

# I'm really hoping that the model I choose will 
# 1. Identify my new features as very important
# 2. Spot instances of my new feature giving wrong results
# 3. Magically identify (and rectify) some comibnation of factors that are common for records in 2.

# As I think through the things that I want, I wonder if kmeans, generating new features at this stage would be the right way to go.
# My kmeans attempt at https://www.kaggle.com/joecooper/experiment-with-kmeans certainly showed that it added value. 
# Recap - I used the predictions from sandpit to generate clusters (or cluster predictions) and then added those to the training data. 
# In this one I'm adding sandpit predictions directly to training data. I believe that (or maybe just hope) that the predictions
# themselves will be more handy than a cluster feature they generate.

# I think combining the 2 together might make task2 and 3 above a little easier for computer to solve. If it was a 2 month competition I would certainly look at that.
# Actually if I had 1 spare day I would look at that. Don't think it would take long to check.

# Anyway - I just randomly choose lgb first and lets see what happens.

# I should also point out that I'm expecting FI to be significant for new features.
# How will that change the gradient of each boost? 
# Certainly a different set of params might be MUCH better - but optuna takes time and I'm only hoping for a little bump up the leaderboard.

# So I just choose another set of lgb params from public notebooks. 
# Not sure having the exact same set as a feature your trained on already is a good idea?
# Again haven't tried, but my instinct say diversify



In [ ]:
%%time

## Moving onto the time killer section
models = [
#     ("lgbm3", LGBMClassifier(**lgbm_params_3)),
    ("lgbm0", LGBMClassifier(**lgbm_params_0)),
#     ("lgbm0a", LGBMClassifier(**lgbm_params_0a)),
#     ("lgbm0b", LGBMClassifier(**lgbm_params_0b)),
#     ("lgbm0c", LGBMClassifier(**lgbm_params_0c)),
#     ("lgbm1", LGBMClassifier(**lgbm_params_1)),
#     ("lgbm1a", LGBMClassifier(**lgbm_params_1a)),
#     ("lgbm1b", LGBMClassifier(**lgbm_params_1b)),
#     ("lgbm1c", LGBMClassifier(**lgbm_params_1c)),
#    ("lgbm1", LGBMClassifier(**lgbm_params_1)),
#    ("lgbm1", LGBMClassifier(**lgbm_params_1)),
#     ("catb0", CatBoostClassifier(**catb_params_0)),
#     ("catb1", CatBoostClassifier(**catb_params_1)),
#     ("xgb0", XGBClassifier(**xgb_params_0)),
#     ("xgb1", XGBClassifier(**xgb_params_1)),
]

oof_pred_tmp = dict()
test_pred_tmp = dict()
scores_tmp = dict()

### Another new addition so that I can see FI
mod_importances = pd.DataFrame()

print ("Start time :", datetime.now() )

kf = StratifiedKFold(n_splits=7, shuffle=True, random_state=12)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    print ("Fold : ", fold, " Start : ", datetime.now())
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]


    for name, model in models:
        start_time = datetime.now()
        print ("model name : ", name, " Start : ", datetime.now())
        if name not in scores_tmp:
            oof_pred_tmp[name] = list()
            oof_pred_tmp["y_valid"] = list()
            test_pred_tmp[name] = list()
            scores_tmp[name] = list()

        if name.startswith("lgbm0"):
            print ("Name starts with lgbm0")
     
            # I think that I will need to go much lower with my new features
            lgb_params['learning_rate'] = 0.003
            model = LGBMClassifier(**lgb_params)
            model.fit( X_train, y_train,
                eval_set=[(X_valid,y_valid)],
                eval_metric='auc',
                early_stopping_rounds=700,        
                verbose=200,
            )
            
            ### messing about with second fits is not needed
            ### I was just messing about with it since I hadn't tried it before. 
            ### Can't say that I was overly impressed
            
            
        
        elif name.startswith("xgb0"):
            print ("Name starts with xgb0")
            
            xgb_params_0['learning_rate'] = 0.02
            pre_model = XGBClassifier(**xgb_params_0)
            pre_model.fit( X_train, y_train,
                eval_set=[(X_valid,y_valid)],
                eval_metric='auc',
                early_stopping_rounds=700,        
                verbose=1000,
            )
            
            xgb_params_0['learning_rate'] *= 0.1
            model = XGBClassifier(**xgb_params_0)
            model.fit(X_train, y_train,
                    eval_set=[(X_valid,y_valid)],
                    eval_metric='auc',
                    early_stopping_rounds=500,
                    verbose=200,
                    xgb_model=pre_model)
            
        elif name.startswith("catb0"):
            print ("Name starts with catb0")
            ### catboost doens't like a boolean target 
            y_train = y_train.astype('float')
            y_valid = y_valid.astype('float')
            model.fit(
                X_train, y_train,
                eval_set=[(X_valid,y_valid)],
                early_stopping_rounds=500,
                verbose=1000)
            
                
        else:
            model.fit(
                X_train, y_train,
                eval_set=[(X_valid,y_valid)],
                early_stopping_rounds=500,
                verbose=1000
            )
        ## end if
        
        # Can't recall where I nicked this, but it is useful
        fi_tmp = pd.DataFrame()
        fi_tmp['feature'] = X_train.columns
        fi_tmp['importance'] = model.feature_importances_
        fi_tmp['fold'] = fold
        fi_tmp['seed'] = 42    ## seed value within params
        mod_importances = mod_importances.append(fi_tmp)
        
        ## This FI code is really handy. It can also be used for handling lopps of seeds as well.
        ## Good practice for LB climbing but overkill for a sandpit
    
        ### Time to get the predictions for oof test data (valid data)
        pred_valid = model.predict_proba(X_valid)[:, -1]
        score = roc_auc_score(y_valid, pred_valid)
        
        scores_tmp[name].append(score)
        oof_pred_tmp[name].extend(pred_valid)
        
        end_time = datetime.now()
        duration = end_time - start_time
        
        f_pred.loc[idx_valid, 'pred'] = model.predict_proba(X_valid)[:, 1]
        
        print(f"Fold: {fold} Model: {name} Score: {score}", "Duration :", duration)

        ### Time to get the predictions for competition test data (df_test descendandts)
        y_hat = model.predict_proba(test_data)[:,1]
        test_pred_tmp[name].append(y_hat)
        
    oof_pred_tmp["y_valid"].extend(y_valid)
        
        
for name, model in models:
    print(f"Average Validation Score | {name}: {np.mean(scores_tmp[name])}")

In [ ]:
f_pred.to_csv("./f_pred_train.csv", index=True)

In [ ]:
# Time to unravel those dictionaries of models and predictions 
test_predictions = pd.DataFrame(  
    {name: np.mean(np.column_stack(test_pred_tmp[name]), axis=1) for name in test_pred_tmp.keys()}
)

# Always handy to save a copy for use later
test_predictions.to_csv("./test_predictions.csv", index=False)

# get the average of all predictions
test_predictions["avg"] = test_predictions.mean(axis=1)

# overwrite target in sample_submission with our new and improved predictions 
avg_submission = sample_submission.copy()
avg_submission["target"] = test_predictions["avg"]
avg_submission.to_csv("./avg_submission.csv", index=False)

In [ ]:
f_pred_test = sample_submission.copy()
f_pred_test['pred'] = test_predictions["avg"]
f_pred_test.to_csv("./f_pred_test.csv", index=True)

In [ ]:
# So that oof_pred_tmp dictionary that we populated during training was so that
# we can get the real validation score across the whole training dataset and not just avergae 
# individual folds. I have once before started a discussion within the competition detailing the 
# importance of this step

oof_predictions = pd.DataFrame({name:oof_pred_tmp[name] for name in oof_pred_tmp.keys()})

# Again Always handy so save a copy for revisiting
oof_predictions.to_csv("./oof_preds.csv", index=False)

# And now we get the all important validation score
y_valid = oof_predictions["y_valid"].copy()
y_hat_blend = oof_predictions.drop(columns=["y_valid"]).mean(axis=1)
score = roc_auc_score(y_valid, y_hat_blend)

print(f"Overall Training Validation Score | Blend: {score}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
order = list(mod_importances.groupby('feature').mean().sort_values('importance', ascending=False).index)

## I want the graph to be readable
for i in range(250):
    order.pop()
## Half embarrassed to put out such lazy code - but be fair ... it is doing the job I want it to.

    

fig = plt.figure(figsize=(16, 16), tight_layout=True)
sns.barplot(x="importance", y="feature", data=mod_importances.groupby('feature').mean().reset_index(), order=order)
plt.title("LGB feature importances")

#  From the original sandpit notebook we see 
#Results tracking
# V3 LGBM0 Overall Training Validation Score | Blend: 0.8569692225223414 with LB score of 0.85633
# V4 LGBM0 Overall Training Validation Score | Blend: 0.8570320102774184 with LB score of 0.85623
# V5 LGBM0 Overall Training Validation Score | Blend: 0.8570190414332162 with LB score of 0.85627
# V6 LGBM1 Overall Training Validation Score | Blend: 0.8571127311293687 with LB score of 0.85633
# V7 XGB0 Overall Training Validation Score | Blend: 0.8570048237818386 with LB score of 0.85645
# V8 XGB1 Overall Training Validation Score | Blend: 0.8568621253215738 with LB score of 0.85621
# V10 CATB0 Overall Training Validation Score | Blend: 0.8566436011399643 with LB score of 0.85597
# V11 LGBM0 Overall Training Validation Score | Blend: 0.8569640619014858 with LB score of 0.85638


# When Running the LGB model the best feature is the xgb output 
# What does that tell us?



In [ ]:
print (f_pred)
print (X[['v7', 'v10', 'v11']])

### So why do I look here ?

# Well I'm only looking at row 0 to start. The original models had predictions of 0.62, 0.59 and 0.64
# Averaging, ensembling, stacking (whatever you want to call that) would normally give a new prediction here of around 0.62
# My new prediction is 0.57
# Now for each row - new results are always closer to 0.5 regardless of direction. Statistically speaking f_pred has lower variance

# So my new results are still the same in 'classification' but they are less confident

# Could we add this new feature into the next version and just keep looping?
# Would pred end up being 1 million rows of 0.5
# Would it help throwing this through a Scaler ?
# Will the stccking benefit of this output be less helpful to the leaderboard?

# In answer to the original question. I look becuase I'm interested! 
# I want to understand and question what is going on. 
# Sometimes looking generates questions that lead to ideas that leads to innvoation.
